<a href="https://colab.research.google.com/github/Likhithluck/Capstone-Project/blob/tarun/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pdf2image
!pip install datasets
!pip install pytesseract
!pip install pdfplumber
!apt-get install tesseract-ocr
!apt-get install libtesseract-dev
!apt-get install poppler-utils # Install poppler-utils which includes pdfinfo


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 98.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-os

In [4]:
import pandas as panda
from huggingface_hub import login
from datasets import load_dataset
from PIL import Image as img
import matplotlib.pyplot as plt
from google.colab import files
import cv2
import numpy as np
import pytesseract
from pdf2image import convert_from_path
import pdfplumber
from google.colab import files
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

In [5]:
from google.colab import userdata
key=userdata.get('hugging_face_key')

In [6]:
login(token=key)

In [ ]:
# dataset1 = load_dataset('mychen76/invoices-and-receipts_ocr_v1')
# dataset2 = load_dataset('mychen76/invoices-and-receipts_ocr_v2')

In [ ]:
# #Basic OCR
# # Preprocessing Function
# def preprocess_image(image):
#     gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
#     denoised = cv2.fastNlMeansDenoising(gray, None, 30, 7, 21)
#     clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8, 8))
#     contrast_enhanced = clahe.apply(denoised)
#     kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
#     cleaned = cv2.morphologyEx(contrast_enhanced, cv2.MORPH_CLOSE, kernel)
#     thresh = cv2.adaptiveThreshold(cleaned, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
#                                    cv2.THRESH_BINARY, 11, 2)
#     return thresh


# def extract_text_from_pdf(pdf_path):
#     images = convert_from_path(pdf_path, dpi=300 ,poppler_path="/usr/bin" )

#     extracted_text = ""

#     for i, image in enumerate(images):
#         processed_image = preprocess_image(image)
#         custom_config = '--psm 6 --oem 3'
#         page_text = pytesseract.image_to_string(processed_image, config=custom_config)
#         extracted_text += f"\n--- Page {i + 1} ---\n{page_text}"

#     print("Extracted Text:\n", extracted_text)


# #Dataset pdf
# pdf_path = "/content/invoice_Yana Sorensen_5434.pdf"
# #OCR extraction
# text=extract_text_from_pdf(pdf_path)
# print(text)

In [ ]:

# def extract_clean_text(pdf_path):
#     clean_text = ""
#     with pdfplumber.open(pdf_path) as pdf:
#         for page_num, page in enumerate(pdf.pages):
#             text = page.extract_text()
#             if text:
#                 clean_text += text.strip() + "\n\n"
#     print(clean_text)
#     with open("clean_text_output.txt", "w", encoding="utf-8") as file:
#         file.write(clean_text)

# # Path to PDF
# pdf_path = "/content/invoice_Yana Sorensen_5434.pdf"

# # Run Extraction
# ocr_pdf_plum=extract_clean_text(pdf_path)


In [15]:
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
import cv2
import numpy as np

#upload file
def upload_pdf():
    uploaded = files.upload()
    pdf_path = next(iter(uploaded))
    return pdf_path

#preprocess the image
def preprocess_image(image):
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
    sharpened = cv2.filter2D(gray, -1, np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]))
    denoised = cv2.fastNlMeansDenoising(sharpened, None, 30, 7, 21)
    clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8, 8))
    contrast_enhanced = clahe.apply(denoised)
    return contrast_enhanced


#OCR Extraction
def extract_text_with_ocr(pdf_path):
    extracted_text = ""
    images = convert_from_path(pdf_path, dpi=300, poppler_path="/usr/bin")

    for i, image in enumerate(images):
        processed_image = preprocess_image(image)
        custom_config = '--psm 3 --oem 3'
        page_text = pytesseract.image_to_string(processed_image, config=custom_config)
        extracted_text += f"--- Page {i + 1} ---\n{page_text.strip()}\n\n"

    return extracted_text

# Table Extraction Using pdfplumber
def extract_tables_from_pdf(pdf_path):
    table_data = ""

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):

            # Adjust table settings
            tables = page.extract_tables(
                table_settings={
                    "snap_tolerance": 10,
                    "join_tolerance": 10,  # Merges fragmented rows
                    "edge_min_length": 50, # Filters out small text artifacts
                    "horizontal_strategy": "lines",  # Try extracting text as lines
                    "vertical_strategy": "lines",  # Strategy to align vertically
                }
            )


            if tables:
                table_data += f"\n--- Tables from Page {page_num + 1} ---\n"
                for table in tables:
                    for row in table:

                        cleaned_row = [str(cell).strip() if cell else '' for cell in row]
                        table_data += " | ".join(cleaned_row) + "\n"
            else:
                print("No table found on this page.")

    return table_data


def extract_data(pdf_path):
    print("(OCR)")
    ocr_text = extract_text_with_ocr(pdf_path)

    print("\nTables")
    table_data = extract_tables_from_pdf(pdf_path)

    return ocr_text, table_data


#Testing
# pdf_path = "/content/invoice_Yana Sorensen_5434.pdf"
pdf_path=upload_pdf()
ocr_text, table_data = extract_data(pdf_path)

print("\nOCR Text\n")
print(ocr_text)
print(table_data)


Saving invoice_Tony Molinari_25881.pdf to invoice_Tony Molinari_25881.pdf
(OCR)

Tables
No table found on this page.

OCR Text

--- Page 1 ---
Superstore INVOICE

# 25881
Daie: Nov 17 2012
Bill To: Ship To: ;
oo Ship Mode: First Class
Tony Molinari Surabaya, Jawa
Timur, Indonesia Balance Due: $4,368.45
Item | Quantity Rate Amount
Novimex Swivel Stool, Black 7 $827.97 $5,795.81
Chairs, Furniture, FUR-CH-5412
Subtotal: $5,795.81
Discount (27%) : $1,564.87
Shipping: $137.51
Total: $4,368.45

Notes:
Thanks for your business!

Terms:
Order ID : IN-2012-TM2149059-41230





In [16]:
# Check the types of ocr_text and table_data
ocr_text_type = type(ocr_text)
table_data_type = type(table_data)

extracted_data = {
    'ocr_text': ocr_text,
    'table_data': table_data
}


In [17]:
extracted_data

{'ocr_text': '--- Page 1 ---\nSuperstore INVOICE\n\n# 25881\nDaie: Nov 17 2012\nBill To: Ship To: ;\noo Ship Mode: First Class\nTony Molinari Surabaya, Jawa\nTimur, Indonesia Balance Due: $4,368.45\nItem | Quantity Rate Amount\nNovimex Swivel Stool, Black 7 $827.97 $5,795.81\nChairs, Furniture, FUR-CH-5412\nSubtotal: $5,795.81\nDiscount (27%) : $1,564.87\nShipping: $137.51\nTotal: $4,368.45\n\nNotes:\nThanks for your business!\n\nTerms:\nOrder ID : IN-2012-TM2149059-41230\n\n',
 'table_data': ''}

In [18]:
import spacy

# NER model
nlp = spacy.load("en_core_web_sm")

def perform_ner(text):
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        entities.append({
            "text": ent.text,
            "label": ent.label_,
            "start_char": ent.start_char,
            "end_char": ent.end_char
        })
    return entities


ner_results = perform_ner(ocr_text)
ner_results

[{'text': '25881', 'label': 'MONEY', 'start_char': 37, 'end_char': 42},
 {'text': 'Daie', 'label': 'ORG', 'start_char': 43, 'end_char': 47},
 {'text': '17 2012', 'label': 'DATE', 'start_char': 53, 'end_char': 60},
 {'text': 'Bill', 'label': 'PERSON', 'start_char': 61, 'end_char': 65},
 {'text': 'Ship Mode', 'label': 'PERSON', 'start_char': 84, 'end_char': 93},
 {'text': 'First', 'label': 'ORDINAL', 'start_char': 95, 'end_char': 100},
 {'text': 'Tony Molinari Surabaya',
  'label': 'PERSON',
  'start_char': 107,
  'end_char': 129},
 {'text': 'Jawa\nTimur',
  'label': 'PERSON',
  'start_char': 131,
  'end_char': 141},
 {'text': 'Indonesia', 'label': 'GPE', 'start_char': 143, 'end_char': 152},
 {'text': '4,368.45', 'label': 'MONEY', 'start_char': 167, 'end_char': 175},
 {'text': '827.97', 'label': 'MONEY', 'start_char': 235, 'end_char': 241},
 {'text': '5,795.81', 'label': 'MONEY', 'start_char': 243, 'end_char': 251},
 {'text': 'FUR', 'label': 'ORG', 'start_char': 271, 'end_char': 274},
 {

In [29]:
import json
import re

def ocr_to_json_with_ner(ocr_text, ner_results):
    # Initialize variables
    invoice_data = {
        "invoice_number": None,
        "date": None,
        "bill_to": None,
        "ship_to": None,
        "ship_mode": None,
        "balance_due": None,
        "items": [],
        "subtotal": None,
        "discount": None,
        "shipping": None,
        "total": None,
        "notes": "",
        "terms": {"order_id": None}
    }

    #Items
    item = []


    #Bill_to
    person_entities = [entity for entity in ner_results if entity["label"] == "PERSON"]
    date_entities = [entity for entity in ner_results if entity["label"] == "DATE"]
    invoice_entities = [entity for entity in ner_results if entity["label"] == "MONEY"]

    # Assign the third PERSON entity to bill_to
    if len(person_entities) >= 1:
        invoice_data["bill_to"] = person_entities[2]["text"]

    # INVOICE NUM
    invoice_pattern = r"(\#\s?\d{1,6})"

    # Find the first match for the invoice number
    match = re.search(invoice_pattern, ocr_text)

    if match:
        invoice_number = match.group(1).strip()
        invoice_data["invoice_number"] = invoice_number

    # Assign ship_to based on bill_to value
    if invoice_data["bill_to"]:
        # Search for the bill_to value in the OCR text
        bill_to_index = ocr_text.find(invoice_data["bill_to"])
        if bill_to_index != -1:
            # Extract the text after the bill_to value
            text_after_bill_to = ocr_text[bill_to_index + len(invoice_data["bill_to"]):]
            # Find the text before "Balance Due"
            balance_due_index = text_after_bill_to.find("Balance Due")
            if balance_due_index != -1:
                # Extract the ship_to value (text between the comma after bill_to and "Balance Due")
                ship_to_value = text_after_bill_to[:balance_due_index].strip()
                # Remove any leading/trailing punctuation
                ship_to_value = ship_to_value.strip(", ")
                # Clean the ship_to value by removing ": " and "\n"
                ship_to_value = ship_to_value.replace(": ", "").replace("\n", " ")
                invoice_data["ship_to"] = ship_to_value

    # Ship_mode
    ship_mode_match = re.search(r"Ship Mode: ([\w\s]+)\n", ocr_text)
    if ship_mode_match:
        ship_mode = ship_mode_match.group(1).strip()
        invoice_data["ship_mode"] = ship_mode


   # Regex pattern to capture various date formats like Nov 17 2012, 17-Nov-2012, or even 2012-11-17
    date_pattern = r"(\b(?:[A-Za-z]{3,9}\s\d{1,2},?\s?\d{4}|\d{1,2}-[A-Za-z]{3}-\d{4}|\d{4}-\d{2}-\d{2})\b)"
    match = re.search(date_pattern, ocr_text)

    if match:
        invoice_data["date"] = match.group(1).strip()

    # Balance_due
    balance_due_match = re.search(r"Balance Due: \$([\d,]+\.\d{2})", ocr_text)
    if balance_due_match:
        balance_due = balance_due_match.group(1).strip()
        invoice_data["balance_due"] = float(balance_due.replace(",", ""))

    # Subtotal
    subtotal_match = re.search(r"Subtotal: \$([\d,]+\.\d{2})", ocr_text)
    if subtotal_match:
      subtotal = subtotal_match.group(1).strip()
      invoice_data["subtotal"] = float(subtotal.replace(",", ""))

    #discount
    discount_match = re.search(r"Discount\s?\(?\d*%?\)?\s?:\s?\$([\d,]+\.\d{2})", ocr_text)
    if discount_match:
        discount = discount_match.group(1).strip()  # Capture just the numeric value part
        invoice_data["discount"] = float(discount.replace(",", ""))

    #Shipping
    shipping_match = re.search(r"Shipping: \$([\d,]+\.\d{2})", ocr_text)
    if shipping_match:
        shipping = shipping_match.group(1).strip()
        invoice_data["shipping"] = float(shipping.replace(",", ""))

    # Total
    total_match = re.search(r"Total: \$([\d,]+\.\d{2})", ocr_text)
    if total_match:
        total = total_match.group(1).strip()
        invoice_data["total"] = float(total.replace(",", ""))


    # Notes
    notes_match = re.search(r"Notes:\s*(.*?)\s*(?=Terms:|$)", ocr_text, re.DOTALL)
    if notes_match:
        notes = notes_match.group(1).strip()
        invoice_data["notes"] = notes

    # Order_ID
    order_id_match = re.search(r"Order ID : (\S+)", ocr_text)
    if order_id_match:
        order_id = order_id_match.group(1).strip()
        invoice_data["terms"]["order_id"] = order_id+date_entities[-1]["text"]


    item_pattern = re.compile(r"([\w\s\-,]+)\s+(\d+)\s+\$([\d,]+\.\d{2})\s+\$([\d,]+\.\d{2})")
    matches = item_pattern.findall(ocr_text)

    for match in matches:
        item_name = match[0].strip()
        item_name = re.sub(r"Quantity Rate Amount\n", "", item_name).strip()
        quantity = int(match[1].strip())
        unit_price = float(match[2].replace(",", "").strip())
        total_price = float(match[3].replace(",", "").strip())

        # Add the item to the list
        item.append({
            "item_name": item_name,
            "quantity": quantity,
            "unit_price": unit_price,
            "total_price": total_price
        })
    #ITEMS
    invoice_data["items"] = item

    return json.dumps({"invoice": invoice_data}, indent=4)



json_output = ocr_to_json_with_ner(ocr_text, ner_results)
print(json_output)

{
    "invoice": {
        "invoice_number": "# 25881",
        "date": "Nov 17 2012",
        "bill_to": "Tony Molinari Surabaya",
        "ship_to": "Jawa Timur, Indonesia",
        "ship_mode": "First Class",
        "balance_due": 4368.45,
        "items": [
            {
                "item_name": "Novimex Swivel Stool, Black",
                "quantity": 7,
                "unit_price": 827.97,
                "total_price": 5795.81
            }
        ],
        "subtotal": 5795.81,
        "discount": 1564.87,
        "shipping": 137.51,
        "total": 4368.45,
        "notes": "Thanks for your business!",
        "terms": {
            "order_id": "IN-2012-TM2149059-4123017 2012"
        }
    }
}
